In [1]:
import cartopy.crs as ccrs
import xarray as xr
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
from numpy.polynomial.polynomial import polyfit
from scipy import signal
from scipy.stats import norm

import matplotlib

import matplotlib.pyplot as plt
import netCDF4 as nc
from matplotlib import pyplot
from matplotlib import colors as mcolors
import matplotlib.gridspec as gridspec
from scipy.stats import norm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import gcm_filters



In [2]:
def compute_spectrum(KE_spectre):
    dx=1/(dx_min.values/1000)
    print(dx_min)
    no=0
    ff_KE_spectre=np.zeros(KE_spectre.shape)
    ff_KE_spectreki0=np.zeros(KE_spectre.shape)
    # len(KE_spectre_20[:,0])
    for j in range(len(KE_spectre[0,:])):
        ff_KE_spectre[:,j], ff_KE_spectreki0[:,j]= signal.welch(KE_spectre[:,j], fs=dx,nperseg=int(len(KE_spectre[:,0])), window='hanning', noverlap=no,nfft=2*int(len(KE_spectre[:,0])-1),  detrend='linear', return_onesided=True, scaling='spectrum')

    mean_f0_KE_spectre = np.nanmean(ff_KE_spectre,axis=1)
    mean_fi0_KE_spectre = np.nanmean(ff_KE_spectreki0,axis=1)
    return mean_f0_KE_spectre, mean_fi0_KE_spectre

In [3]:
Tfile_AGRIF_qco = xr.open_dataset('/data/vdi/tbrivoal/RUNS_DATA/EXP02_AGRIF_finaldomain_bathycorrected_qco_boost2_noslip/eNEATL36_1d_KE2.nc', chunks={'time_counter': 50})
Tfile_TWIN = xr.open_dataset('/data/vdi/tbrivoal/RUNS_DATA/eNEATL36_trunk_r4_2-RC_r15113_IFS_EXP02_2017_2018_AGRIFTWIN_TRUE/eNEATL36_1d_KE2.nc', chunks={'time_counter': 50})

Coordfile = xr.open_dataset('/data/vdi/tbrivoal/PRE_PROCESSING_IMMERSE/BATHY/DOMAINcfgs/domain_cfg_init.nc', drop_variables={"x", "y",})

output_dir_AGRIF_qco='/data/vdi/tbrivoal/RUNS_DATA/EXP02_AGRIF_finaldomain_bathycorrected_qco_boost2_noslip/'
output_dir_TWIN='/data/vdi/tbrivoal/RUNS_DATA/eNEATL36_trunk_r4_2-RC_r15113_IFS_EXP02_2017_2018_AGRIFTWIN_TRUE/'

In [4]:

T_AGRIF_qco=Tfile_AGRIF_qco.KE2.squeeze().sel(time_counter=slice('2017-01-04', '2018-08-02')) *100 *100 *0.5# .mean(dim='time_counter') *100 *100 * 0.5 
T_TWIN=Tfile_TWIN.KE2.squeeze().sel(time_counter=slice('2017-01-04', '2018-08-02')) *100 *100 *0.5# .mean(dim='time_counter') *100 *100 * 0.5 

wet_mask_AGRIF_qco = xr.where(~xr.ufuncs.isnan(T_AGRIF_qco[:,:,:].squeeze()), 1, 0)
wet_mask_TWIN = xr.where(~xr.ufuncs.isnan(T_TWIN[:,:,:].squeeze()), 1, 0)

dxe = Coordfile.e1t.squeeze()
dye = Coordfile.e2t.squeeze()
dxn = Coordfile.e1v.squeeze()
dyn = Coordfile.e2v.squeeze()
dx_min = min(dxe.min(),dye.min(),dxn.min(),dyn.min())
area = dxe * dye
T_AGRIF_qco_tmp=T_AGRIF_qco[0,:,:].drop("time_counter")
wet_mask_tmp=wet_mask_AGRIF_qco[0,:,:].drop("time_counter")
wet_mask_tmp.compute()
T_AGRIF_qco.compute()


<xarray.DataArray 'KE2' (time_counter: 416, y: 1894, x: 1294)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan, 44.139507 , 44.12184  , ...,        nan,
                nan,        nan],
        [       nan, 45.339577 , 30.602623 , ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan, 37.480484 , 38.0677   , ...,        nan,
                nan,        nan],
        [       nan, 38.573727 , 41.07554  , ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan, 28.128887 , 26.111048 , ...,        nan,
                nan,        nan],
        [       nan, 29.634148 , 35.75762  , ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2017-01-05 ... 2018-02-24
    nav_lon       (y, x) float32 -19.92 -19.89 -19.86 -19.83 ... -1.0 -1.0 -1.0
    nav_lat       (y, x) float32 26.13 26.13 26.13 26.13 ... -1.0 -1.0 -1.0 -1.0
    depthu        float32 0.494
Dimensions without coordinates: y, x

In [5]:
# T_AGRIF_qco_coarsen_tmp = T_AGRIF_qco_tmp.squeeze().astype(np.float64).coarsen(x=coarsen_fac, y=coarsen_fac, boundary="trim").mean()
# area_coarsen = area.astype(np.float64).coarsen(x=coarsen_fac, y=coarsen_fac, boundary="trim").mean()
# wet_mask_tmp_coarsen = wet_mask_tmp.astype(np.float64).coarsen(x=coarsen_fac, y=coarsen_fac, boundary="trim").mean()
# wet_mask_tmp_coarsen_tmp = wet_mask_tmp_coarsen.values
# wet_mask_tmp_coarsen_tmp[0,:]=0.
# wet_mask_tmp_coarsen_tmp[-1,:]=0.
# wet_mask_tmp_coarsen_tmp[:,0]=0.
# wet_mask_tmp_coarsen_tmp[:,-1]=0.


# wet_mask_tmp_coarsen = xr.DataArray(wet_mask_tmp_coarsen_tmp, coords=wet_mask_tmp_coarsen.coords  )

In [6]:
specs20 = {
    'filter_scale': 20,
    'dx_min': 1,
    'filter_shape': gcm_filters.FilterShape.GAUSSIAN
}

specs80 = {
    'filter_scale': 80,
    'dx_min': 1,
    'filter_shape': gcm_filters.FilterShape.GAUSSIAN
}


specs50 = {
    'filter_scale': 50,
    'dx_min': 1,
    'filter_shape': gcm_filters.FilterShape.GAUSSIAN
}

specs130 = {
    'filter_scale': 130,
    'dx_min': 1,
    'filter_shape': gcm_filters.FilterShape.GAUSSIAN
}
#############################################################
filter_tripolar_regular_with_land_20 = gcm_filters.Filter(
    **specs20,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=3,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_tmp.astype(np.float64)}
)
#############################################################
filter_tripolar_regular_with_land_80 = gcm_filters.Filter(
    **specs80,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=3,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_tmp.astype(np.float64)}
)

#############################################################
filter_tripolar_regular_with_land_50 = gcm_filters.Filter(
    **specs50,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=3,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_tmp.astype(np.float64)}
)

#############################################################
filter_tripolar_regular_with_land_130 = gcm_filters.Filter(
    **specs130,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=7,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_tmp.astype(np.float64)}
)

# #############################################################
# filter_tripolar_regular_with_land_130 = gcm_filters.Filter(
#     **specs130,
#     grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
#     n_iterations=5,  # number of constituent filters

#     grid_vars={'area': area_coarsen.astype(np.float64), 'wet_mask': wet_mask_tmp_coarsen.astype(np.float64)}
# )

#############################################################
#############################################################
#############################################################
#############################################################

filter_tripolar_regular_with_land_20_AGRIF_qco = gcm_filters.Filter(
    **specs20,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=3,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_AGRIF_qco.astype(np.float64)}
)
#############################################################
filter_tripolar_regular_with_land_80_AGRIF_qco = gcm_filters.Filter(
    **specs80,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=3,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_AGRIF_qco.astype(np.float64)}
)

#############################################################
filter_tripolar_regular_with_land_50_AGRIF_qco = gcm_filters.Filter(
    **specs50,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=3,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_AGRIF_qco.astype(np.float64)}
)

#############################################################
filter_tripolar_regular_with_land_130_AGRIF_qco = gcm_filters.Filter(
    **specs130,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=7,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_AGRIF_qco.astype(np.float64)}
)


#############################################################
#############################################################
#############################################################
#############################################################

filter_tripolar_regular_with_land_20_TWIN = gcm_filters.Filter(
    **specs20,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=3,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_TWIN.astype(np.float64)}
)
#############################################################
filter_tripolar_regular_with_land_80_TWIN = gcm_filters.Filter(
    **specs80,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=3,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_TWIN.astype(np.float64)}
)

#############################################################
filter_tripolar_regular_with_land_50_TWIN = gcm_filters.Filter(
    **specs50,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=3,  # number of constituent filters

    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_TWIN.astype(np.float64)}
)
coarsen_fac=4
#############################################################
filter_tripolar_regular_with_land_130_TWIN = gcm_filters.Filter(
    **specs130,
    grid_type=gcm_filters.GridType.TRIPOLAR_REGULAR_WITH_LAND_AREA_WEIGHTED,
    n_iterations=7,  # number of constituent filters
    grid_vars={'area': area.astype(np.float64), 'wet_mask': wet_mask_TWIN.astype(np.float64)}
)





In [7]:

# KE_LS_20=filter_tripolar_regular_with_land_20.apply(T_AGRIF_qco_tmp.squeeze().astype(np.float64), dims=['y', 'x'])
# KE_LS_80=filter_tripolar_regular_with_land_80.apply(T_AGRIF_qco_tmp.squeeze().astype(np.float64), dims=['y', 'x'])
# KE_LS_50=filter_tripolar_regular_with_land_50.apply(T_AGRIF_qco_tmp.squeeze().astype(np.float64), dims=['y', 'x'])
# KE_LS_130=filter_tripolar_regular_with_land_130.apply(T_AGRIF_qco_tmp.squeeze().astype(np.float64), dims=['y', 'x'])


# KE_LS_20_nomsk = KE_LS_20.where(~np.isnan(KE_LS_20.values),0).values
# KE_LS_50_nomsk = KE_LS_50.where(~np.isnan(KE_LS_50.values),0).values
# KE_LS_80_nomsk = KE_LS_80.where(~np.isnan(KE_LS_80.values),0).values
# KE_LS_130_nomsk = KE_LS_130.where(~np.isnan(KE_LS_130.values),0).values

# KE_noflt = T_AGRIF_qco_tmp.squeeze().where(~np.isnan(T_AGRIF_qco_tmp.squeeze()),0).values
# x_20, y_20 = compute_spectrum(KE_noflt - KE_LS_20_nomsk)

# x_50, y_50 = compute_spectrum(KE_noflt - KE_LS_50_nomsk)
# x_80, y_80 = compute_spectrum(KE_noflt - KE_LS_80_nomsk)
# x_130, y_130 = compute_spectrum(KE_noflt - KE_LS_130_nomsk)

# x_noflt, y_noflt = compute_spectrum(KE_noflt)

In [8]:
# dx=1/(dx_min.values/1000)
# from matplotlib.ticker import ScalarFormatter, FormatStrFormatter

# wnmin=1/(int(len(KE_LS_20_nomsk[:,0])) * (1/dx))
# wnmax=dx
# wlmax=(int(len(KE_LS_20_nomsk[:,0])) * (1/dx)) # 1mois #(1/freqmax)/3600
# ax = plt.subplot(111)
# ax.plot(1/x_20,y_20 , 'b', lw=2, label ='20')
# ax.plot(1/x_50,y_50 , 'r', lw=2, label ='50')
# ax.plot(1/x_80,y_80, 'g', lw=2, label ='80')
# ax.plot(1/x_130,y_130, 'brown', lw=2, label ='130')

# ax.plot(1/x_noflt,y_noflt, 'k', lw=2, label ='#nofilter')

# ax.axvline(
#     80*(1/dx)*1/np.sqrt(2),
#     color="g",
#     linewidth=1,
# )

# ax.axvline(
#     50*(1/dx)*1/np.sqrt(2),
#     color="r",
#     linewidth=1,
# )

# ax.axvline(
#     20*(1/dx)*1/np.sqrt(2),
#     color="b",
#     linewidth=1,
# )


# ax.axvline(
#     130*(1/dx)*1/np.sqrt(2),
#     color="brown",
#     linewidth=1,
# )
# print(100*(1/dx)*1/np.sqrt(2))

# ax.set_yscale('log')

# ax.set_xscale('log')
# ax.legend()
# ax.set_xlim(0,800)
# plt.grid(which='minor')
# ax.xaxis.set_major_formatter(FormatStrFormatter('%0.0f'))
# ax.set_ylabel('power') # regex: ($10log10$)
# ax.set_xlabel('Wavelength (km)' )

# plt.savefig('spectre_separation_dechelles.png')

In [9]:

KE_LS_20_AGRIF_qco=filter_tripolar_regular_with_land_20_AGRIF_qco.apply(T_AGRIF_qco.squeeze().astype(np.float64), dims=['y', 'x'])
KE_LS_50_AGRIF_qco=filter_tripolar_regular_with_land_50_AGRIF_qco.apply(T_AGRIF_qco.squeeze().astype(np.float64), dims=['y', 'x'])
KE_LS_80_AGRIF_qco=filter_tripolar_regular_with_land_80_AGRIF_qco.apply(T_AGRIF_qco.squeeze().astype(np.float64), dims=['y', 'x'])
KE_LS_130_AGRIF_qco=filter_tripolar_regular_with_land_130_AGRIF_qco.apply(T_AGRIF_qco.squeeze().astype(np.float64), dims=['y', 'x'])


KE_LS_20_TWIN=filter_tripolar_regular_with_land_20_TWIN.apply(T_TWIN.squeeze().astype(np.float64), dims=['y', 'x'])
KE_LS_50_TWIN=filter_tripolar_regular_with_land_50_TWIN.apply(T_TWIN.squeeze().astype(np.float64), dims=['y', 'x'])
KE_LS_80_TWIN=filter_tripolar_regular_with_land_80_TWIN.apply(T_TWIN.squeeze().astype(np.float64), dims=['y', 'x'])
KE_LS_130_TWIN=filter_tripolar_regular_with_land_130_TWIN.apply(T_TWIN.squeeze().astype(np.float64), dims=['y', 'x'])




KE_LS_20_AGRIF_qco = KE_LS_20_AGRIF_qco.rename('KE2')
KE_LS_50_AGRIF_qco = KE_LS_50_AGRIF_qco.rename('KE2')
KE_LS_80_AGRIF_qco = KE_LS_80_AGRIF_qco.rename('KE2')
KE_LS_130_AGRIF_qco = KE_LS_130_AGRIF_qco.rename('KE2')

KE_LS_20_TWIN = KE_LS_20_TWIN.rename('KE2')
KE_LS_50_TWIN = KE_LS_50_TWIN.rename('KE2')
KE_LS_80_TWIN = KE_LS_80_TWIN.rename('KE2')
KE_LS_130_TWIN = KE_LS_130_TWIN.rename('KE2')


In [ ]:
KE_LS_20_AGRIF_qco.to_netcdf(str(output_dir_AGRIF_qco)+'KE_filtered_gcmfilter_20_new.nc' )
del(KE_LS_20_AGRIF_qco)
print("1 OK")

KE_LS_50_AGRIF_qco.to_netcdf(str(output_dir_AGRIF_qco)+'KE_filtered_gcmfilter_50_new.nc' )
del(KE_LS_50_AGRIF_qco)
print("2 OK")

KE_LS_130_AGRIF_qco.to_netcdf(str(output_dir_AGRIF_qco)+'KE_filtered_gcmfilter_130_new.nc' )
del(KE_LS_130_AGRIF_qco)
print("3 OK")

KE_LS_20_TWIN.to_netcdf(str(output_dir_TWIN)+'KE_filtered_gcmfilter_20_new.nc' )
del(KE_LS_20_TWIN)
print("1 OK")

KE_LS_50_TWIN.to_netcdf(str(output_dir_TWIN)+'KE_filtered_gcmfilter_50_new.nc' )
del(KE_LS_50_TWIN)
print("2 OK")

KE_LS_130_TWIN.to_netcdf(str(output_dir_TWIN)+'KE_filtered_gcmfilter_130_new.nc' )
del(KE_LS_130_TWIN)
print("3 OK")

In [ ]:
KE_noflt = T_AGRIF_qco_tmp.squeeze().where(~np.isnan(T_AGRIF_qco_tmp.squeeze()),0).values

x_20, y_20 = compute_spectrum(KE_noflt - KE_LS_20_nomsk)

x_50, y_50 = compute_spectrum(KE_noflt - KE_LS_50_nomsk)
x_80, y_80 = compute_spectrum(KE_noflt - KE_LS_80_nomsk)
x_130, y_130 = compute_spectrum(KE_noflt - KE_LS_130_nomsk)

x_noflt, y_noflt = compute_spectrum(KE_noflt)